In [78]:
%pylab nbagg
from tvb.simulator.lab import *
from tvb.datatypes import time_series
from tvb.basic.config import settings
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [4]:
jrm = models.JansenRit(mu=0., v0=6.)

phi_n_scaling = (jrm.a * jrm.A * (jrm.p_max-jrm.p_min) * 0.5 )**2 / 2.
sigma         = numpy.zeros(6) 
sigma[3]      = phi_n_scaling

# the other aspects of the simulator are standard
sim = simulator.Simulator(
    model=jrm,
    connectivity=connectivity.Connectivity(load_default=True),
    coupling=coupling.SigmoidalJansenRit(a=10.0),
    integrator=integrators.HeunStochastic(dt=2 ** -4, noise=noise.Additive(nsig=sigma)),
    monitors=monitors.TemporalAverage(period=2 ** -2),
    simulation_length=1e3,
).configure()

# run it
(time_array, data_array), = sim.run()

In [7]:
series_of_time = time_series.TimeSeries(storage = False)
series_of_time.data = data_array
series_of_time.time = time_array

In [9]:
series_of_time.configure()

In [50]:
time_min = np.min(series_of_time.time)
time_max = np.max(series_of_time.time)
time_mean = np.mean(series_of_time.time)

data_min = np.min(series_of_time.data)
data_max = np.max(series_of_time.data)
data_mean = np.mean(series_of_time.data)

In [88]:
import h5py

f = h5py.File("C:\\WORK\\TVB\\simmulation\\2017-09-27_15-52_TimeSeriesRegion2.h5", 'w')

time_size = len(time_array)
data_size = len(data_array)

time_set = f.create_dataset("time",data=time_array, maxshape=(None,))
data_set = f.create_dataset("data",data=data_array)

time_set.attrs['TVB_Minimum'] = time_min
time_set.attrs['TVB_Maximum'] = time_max
time_set.attrs['TVB_Mean'] = time_mean

data_set.attrs['TVB_Minimum'] = data_min
data_set.attrs['TVB_Maximum'] = data_max
data_set.attrs['TVB_Mean'] = data_mean




In [89]:
f.attrs['TVB_Connectivity'] = "f6be362b-5bb4-11e5-8b0d-a45e60e5b22f"
f.attrs['TVB_Create_date'] = "datetime:"+str(datetime.datetime.now())
f.attrs['TVB_Data_version'] = "4"
f.attrs['TVB_Gid'] = generate_guid()
f.attrs['TVB_Has_surface_mapping'] = _serialize_value(series_of_time.has_surface_mapping)
f.attrs['TVB_Has_volume_mapping'] = _serialize_value(series_of_time.has_volume_mapping)
f.attrs['TVB_Invalid'] = "false"
f.attrs['TVB_Is_nan'] = np.isnan(data_array).any()
f.attrs['TVB_Labels_dimensions'] = str(_serialize_value(series_of_time.labels_dimensions))
f.attrs['TVB_Labels_ordering'] = _serialize_value(series_of_time.labels_ordering)
f.attrs['TVB_Length_1d'] = series_of_time.length_1d
f.attrs['TVB_Length_2d'] = series_of_time.length_2d
f.attrs['TVB_Length_3d'] = series_of_time.length_3d
f.attrs['TVB_Length_4d'] = series_of_time.length_4d
f.attrs['TVB_Module'] = "tvb.datatypes.time_series"
f.attrs['TVB_Nr_dimensions'] = series_of_time.nr_dimensions
f.attrs['TVB_Region_mapping'] = ""
f.attrs['TVB_Sample_period'] = series_of_time.sample_period
f.attrs['TVB_Sample_period_unit'] = series_of_time.sample_period_unit
f.attrs['TVB_Sample_rate'] = series_of_time.sample_rate
f.attrs['TVB_Start_time'] = series_of_time.start_time
f.attrs['TVB_State'] = "INTERMEDIATE"
f.attrs['TVB_Subject'] = "John Doe"
f.attrs['TVB_Title'] = series_of_time.title
f.attrs['TVB_Type'] = "TimeSeriesRegion"
f.attrs['TVB_User_tag_1'] = "You can type any text you want here" 
f.attrs['TVB_User_tag_2'] = ""
f.attrs['TVB_User_tag_3'] = ""
f.attrs['TVB_User_tag_4'] = ""
f.attrs['TVB_User_tag_5'] = ""
f.attrs['TVB_Visible'] = "bool:True"

In [90]:
f.close()

In [66]:
BOOL_VALUE_PREFIX = "bool:"
DATETIME_VALUE_PREFIX = "datetime:"
DATE_TIME_FORMAT = '%Y-%m-%d %H:%M:%S.%f'
def date2string(date_input, complex_format=True, date_format=None):
    """Convert date into string, after internal format"""
    if date_input is None:
        return "None"

    if date_format is not None:
        return date_input.strftime(date_format)

    if complex_format:
        return date_input.strftime(COMPLEX_TIME_FORMAT)
    return date_input.strftime(SIMPLE_TIME_FORMAT)


    
def _serialize_value(value):
    """
    This method takes a value which will be stored as metadata and 
    apply some transformation if necessary
      
    :param value: value which is planned to be stored
    :returns:  value to be stored
        
     """
    if value is None:
        return ''
    # Force unicode strings to simple strings.
    if isinstance(value, unicode):
        return str(value)
    # Transform boolean to string and prefix it
    elif isinstance(value, bool):
        return BOOL_VALUE_PREFIX + str(value)
    else:
        return value


In [83]:
import uuid
def generate_guid():
    """ 
    Generate new Global Unique Identifier.
    This identifier should be unique per each station, 
    and unique for different machines.
    """
    return str(uuid.uuid1())
